In [2]:
from time import sleep

import requests
import json
import pandas as pd
from collections import defaultdict

In [4]:
# Read the CSV file
file_path = './Application_types.csv'
df = pd.read_csv(file_path)

# Convert the 'Keyword' column to a list
topic_list = df['Keyword'].tolist()

# Display the list
print(topic_list)


def write_json_data(fileName,data):
    with open(fileName, 'w') as file:
        json.dump(data, file, indent=4)
    
def append_json_data(fileName, data):
    # Try to read existing data if the file exists
    try:
        with open(fileName, 'r') as file:
            existing_data = json.load(file)
    except FileNotFoundError:
        existing_data = []  # Initialize as an empty list if the file doesn't exist
    
    # Append new data to existing data
    existing_data.append(data)

    # Write the updated data back to the file
    with open(fileName, 'w') as file:
        json.dump(existing_data, file, indent=4)

['Dashboard', 'CMS', 'E-commerce', 'Portfolio', 'Blog', 'Chatbot', 'Social Network', 'Messaging', 'Forum', 'Community', 'News', 'Wiki', 'File Manager', 'Analytics', 'Project Management', 'Task Manager', 'Calendar', 'Notes', 'Kanban', 'ERP', 'CRM', 'POS', 'Inventory', 'To-do', 'Video Streaming', 'Music Player', 'Photo Gallery', 'Weather', 'Finance', 'Budget Tracker', 'Expense Tracker', 'Banking', 'Travel', 'Booking', 'Reservation', 'Hotel Management', 'Real Estate', 'Job Portal', 'Classroom', 'Learning Management System', 'Online Exam', 'Survey', 'Poll', 'E-learning', 'Quiz', 'Recipe', 'Cookbook', 'Portfolio', 'Resume Builder', 'Event Management', 'Gaming', 'Game Tracker', 'Game Leaderboard', 'VR', 'AR', '3D Modeling', 'Fitness', 'Workout Tracker', 'Health Tracker', 'Diet Planner', 'Meal Planner', 'Meditation', 'Habit Tracker', 'Productivity', 'Chat', 'Collaboration', 'Help Desk', 'Customer Support', 'Feedback', 'Documentation', 'Startup', 'Portfolio', 'Map', 'Geo-location', 'Travel Gui

In [5]:
url = 'https://api.github.com/search/repositories'
params = {
    'q': f'(language:JavaScript OR language:TypeScript) dashboard stars:>50',
    'sort': 'stars',
    'order': 'desc',
    'per_page': 1,  
    'page': 1
}

response = requests.get(url,  params=params)
    
print(response)

<Response [200]>


In [8]:
def get_github_projects_of_topic(num_results,topic, token=None):
    headers = {'Authorization': f'token {token}'} if token else {}
    url = 'https://api.github.com/search/repositories'
    params = {
        'q': f'(language:JavaScript OR language:TypeScript) {topic} stars:>50',
        'sort': 'stars',
        'order': 'desc',
        'per_page': num_results,  
        'page': 1
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        write_json_data('./topic_data.json',data)
        # pretty_json = json.dumps(data['items'], indent=4)
        # print(pretty_json)
        projects = [{'id': item['id'], 'name': item['name'],'owner': item['owner']['login'], 'watchers': item['watchers_count'],
                     'created_at': item['created_at'],'updated_at': item['updated_at']
                     ,'size': item['size'],'stars': item['stargazers_count']
                     ,'open_issues_count': item['open_issues_count']
                     ,'forks': item['forks']} for item in data['items']]
        # df = pd.DataFrame(projects)
        return projects

    else:
        print(f"Error: {response.status_code}")
        return None
    
topic_dict = {}
num_topics = 10
for i in range(num_topics):
    currentTopic =  topic_list[i]
    projects = get_github_projects_of_topic(num_results=10, topic=currentTopic)
    if projects is not None:
        topic_dict[currentTopic] = projects

print(topic_dict)








{'Dashboard': [{'id': 88464704, 'name': 'vue-element-admin', 'owner': 'PanJiaChen', 'watchers': 87907, 'created_at': '2017-04-17T03:35:49Z', 'updated_at': '2024-11-07T06:04:04Z', 'size': 15433, 'stars': 87907, 'open_issues_count': 1351, 'forks': 30462}, {'id': 15111821, 'name': 'grafana', 'owner': 'grafana', 'watchers': 64711, 'created_at': '2013-12-11T15:59:56Z', 'updated_at': '2024-11-07T06:00:49Z', 'size': 1169352, 'stars': 64711, 'open_issues_count': 4208, 'forks': 12113}, {'id': 43441403, 'name': 'strapi', 'owner': 'strapi', 'watchers': 63676, 'created_at': '2015-09-30T15:34:48Z', 'updated_at': '2024-11-07T03:19:44Z', 'size': 534844, 'stars': 63676, 'open_issues_count': 910, 'forks': 8101}, {'id': 20619036, 'name': 'pi-hole', 'owner': 'pi-hole', 'watchers': 49114, 'created_at': '2014-06-08T15:02:55Z', 'updated_at': '2024-11-07T02:41:27Z', 'size': 8178, 'stars': 49114, 'open_issues_count': 36, 'forks': 2696}, {'id': 15428480, 'name': 'AdminLTE', 'owner': 'ColorlibHQ', 'watchers': 4

In [9]:
all_projects = []

for topic, projects in topic_dict.items():
    for project in projects:
        # Add the topic as a new field in each project dictionary
        project['topic'] = topic
        all_projects.append(project)

# Convert the list of all projects with topics into a DataFrame
df = pd.DataFrame(all_projects)

# Save the DataFrame to a CSV file
df.to_csv('github_projects_with_topics.csv', index=False)

print("Data saved to github_projects_with_topics.csv")

error_from = ""

Data saved to github_projects_with_topics.csv


,id,name,owner,watchers,created_at,updated_at,size,stars,open_issues_count,forks
0,28457823,freeCodeCamp,freeCodeCamp,405050,2014-12-24T17:49:19Z,2024-11-04T06:28:05Z,473268,405050,244,37992
1,10270250,react,facebook,228937,2013-05-24T16:15:54Z,2024-11-04T06:40:39Z,633391,228937,869,46842
2,11730342,vue,vuejs,207957,2013-07-29T03:24:51Z,2024-11-04T05:30:38Z,32152,207957,602,33677
3,88011908,project-based-learning,practical-tutorials,202947,2017-04-12T05:07:46Z,2024-11-04T06:40:54Z,438,202947,186,26484
4,126577260,javascript-algorithms,trekhleb,188244,2018-03-24T07:47:04Z,2024-11-04T05:43:52Z,13398,188244,356,30247


In [ ]:
from collections import defaultdict

def get_Sbom_of_projects(repo_df, token=None):
    headers = {'Authorization': f'token {token}'} if token else {}
    url = 'https://api.github.com/repos/'
    owner = repo_df['owner']
    repo = repo_df['name']
    url = url + f'{owner}/{repo}/dependency-graph/sbom'

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        # write_json_data('./sbom.json',data)
        packages = data['sbom']['packages']
         # Extract npm packages with name and version
        pkg_dict = defaultdict(dict)
        for package in packages:
            if package['externalRefs'][0]['referenceLocator'].startswith('pkg:npm'):
                try:
                    if len(package['versionInfo'])>0:
                        if 'versionInfo' in package:
                            if package['name'].startswith('npm:'):
                                sbom_dict[repo][package['name'][4:]] = package['versionInfo']
                                pkg_dict[repo][package['name'][4:]] = package['versionInfo']
                            else:
                                sbom_dict[repo][package['name']] = package['versionInfo']
                                pkg_dict[repo][package['name']] = package['versionInfo']
                except:
                    print(package)
        append_json_data('./append_dependencies.json',pkg_dict)
    else:
        print(f"Error: {response.status_code}")
        return None

sbom_dict = defaultdict(dict)
flag=False
for topic,projects in topic_dict.items():
    if(topic=='Blog'):
        flag=True
    if(flag):
        for project in projects:
            get_Sbom_of_projects(project)
            sleep(10)
        print(f'{topic} done')
        sleep(60)


# write_json_data('./project_packages.json',sbom_dict)
# print(sbom_dict)